In [87]:
import re
from typing import List, Union, Tuple
from nltk.corpus import stopwords as stpwrds
from sklearn.model_selection import train_test_split
import time
from os.path import isfile
import numpy as np
# Webscraping
from bs4 import BeautifulSoup
from requests import get
import seaborn as sns
import pandas as pd
import nltk
import unicodedata
import nltk.sentiment
from importlib import reload

import os

import matplotlib.pyplot as plt
import json
from typing import Dict, List, Optional, Union, cast
import requests
import prepare as p
from nltk.tokenize.toktok import ToktokTokenizer

from scipy import stats
import acquire as w

In [102]:
df = w.get_michelin_pages()

In [81]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6780 entries, 0 to 6779
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   name                     6780 non-null   object 
 1   address                  6780 non-null   object 
 2   location                 6780 non-null   object 
 3   price                    6726 non-null   object 
 4   cuisine                  6780 non-null   object 
 5   longitude                6780 non-null   float64
 6   latitude                 6780 non-null   float64
 7   phone_number             6649 non-null   float64
 8   url                      6780 non-null   object 
 9   website_url              5542 non-null   object 
 10  award                    6780 non-null   object 
 11  facilities_and_services  6732 non-null   object 
 12  data                     6780 non-null   object 
dtypes: float64(3), object(10)
memory usage: 688.7+ KB


In [103]:
reload(p)
train, validate, test= p.prepare_michelin(df)

In [104]:
train

,name,address,location,price,cuisine,longitude,latitude,url,award,facilities_and_services,data,price_level,city,country,clean,lemmatized,sentiment,word_count
6345,gasthaus ödenturm,"am ödenturm 11, cham, 93413, germany","cham, germany",€,"country cooking, mediterranean cuisine",12.703851,49.210919,https://guide.michelin.com/en/bayern/cham/rest...,bib gourmand,"car park,credit card / debit card accepted,gre...","as if straight out of a picture book, this bea...",1,cham,germany,straight picture book beautiful inn stands edg...,straight picture book beautiful inn stand edge...,0.9136,38
6427,weinhaus anker,"obertorstraße 13, marktheidenfeld, 97828, germany","marktheidenfeld, germany",€,"french, country cooking",9.600301,49.845621,https://guide.michelin.com/en/bayern/marktheid...,bib gourmand,"american express credit card,car park,credit c...",a restaurant that is steeped in tradition and ...,1,marktheidenfeld,germany,steeped tradition run great commitment lovely ...,steeped tradition run great commitment lovely ...,0.9186,35
5217,yang bao bao (nanzih),"106 chaoming road, nanzih district, kaohsiung,...","kaohsiung, taipei & taichung",$,dumplings,120.328050,22.724160,https://guide.michelin.com/en/kaohsiung-region...,bib gourmand,"air conditioning,car park,cash only","in 1993, this dumpling shop began life as a st...",1,kaohsiung,taipei & taichung,1993 dumpling shop began life street stall loc...,1993 dumpling shop began life street stall loc...,0.6808,49
3858,le bistrot gastronómico,"temprado 12, castelló de la plana, 12002, spain","castelló de la plana, spain",€€,fusion,-0.037028,39.983588,https://guide.michelin.com/en/comunidad-valenc...,bib gourmand,"air conditioning,american express credit card,...","an unpretentious, friendly and informal restau...",2,castelló de la plana,spain,unpretentious friendly informal whose aim prov...,unpretentious friendly informal whose aim prov...,0.9607,49
124,fat duck,"high street, bray, sl6 2aq, united kingdom","bray, united kingdom",££££,creative,-0.701753,51.507858,https://guide.michelin.com/en/windsor-and-maid...,3 michelin stars,"air conditioning,american express credit card,...",this iconic restaurant celebrated its 25-year ...,4,bray,united kingdom,iconic celebrated 25year anniversary 2021 intr...,iconic celebrated 25year anniversary 2021 intr...,0.9774,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4492,trattoria cibrèo-cibreino,"via dei macci 122 r, florence, 50122, italy","florence, italy",€,"country cooking, traditional cuisine",11.266419,43.771244,https://guide.michelin.com/en/toscana/firenze/...,bib gourmand,"air conditioning,american express credit card,...",this trattoria is named after the famous cibre...,1,florence,italy,trattoria named famous cibreo typical stew flo...,trattoria named famous cibreo typical stew flo...,0.8126,33
2145,the white room by jacob jan boerma,"dam 9, amsterdam, 1012 js, netherlands","amsterdam, netherlands",€€€€,modern cuisine,4.894229,52.372726,https://guide.michelin.com/en/noord-holland/am...,1 michelin star,"air conditioning,american express credit card,...",you can imagine yourself as an austrian prince...,4,amsterdam,netherlands,imagine austrian prince princess white gold ro...,imagine austrian prince princess white gold ro...,0.9643,28
913,coto de quevedo,"paraje tejeras viejas, torre de juan abad, 133...","torre de juan abad, spain",€€€,traditional cuisine,-3.049166,38.579954,https://guide.michelin.com/en/castilla-la-manc...,1 michelin star,"air conditioning,car park,credit card / debit ...",if you’re looking for somewhere to eat with a ...,3,torre de juan abad,spain,youre looking somewhere eat solid reputation l...,youre looking somewhere eat solid reputation l...,0.9393,78
4738,kyoyang siksa,"238 itaewon-ro, yongsan-gu, seoul, south korea","seoul, south korea",₩₩,barbecue,126.999940,37.535920,https://guide.michelin.com/en/seoul-capital-ar...,bib gourmand,"counter dining,valet parking",sapporo-style lamb barbecue i

In [107]:
def get_stats_ttest(df):
    '''Function returns statistical T test'''
    One_Star = df[df.award == '1 michelin star']
    Two_Star = df[df.award == '2 michelin stars']
    stat, pval = stats.levene(One_Star.sentiment,
                              Two_Star.sentiment)
    alpha = 0.05
    if pval < alpha:
        variance = False

    else:
        variance = True    
    t_stat, p_val = stats.ttest_ind(One_Star.sentiment,
                                    Two_Star.sentiment,
                                    equal_var=True, random_state=123)

    print(f't_stat= {t_stat}, p_value= {p_val/2}')
    print('-----------------------------------------------------------')
    if (p_val/2) < alpha:
        print(f'We reject the null Hypothesis')
    else:
        print(f'We fail to reject the null Hypothesis.')

In [108]:
get_stats_ttest(train)

t_stat= 0.4534080508004802, p_value= 0.3251547361749879
-----------------------------------------------------------
We fail to reject the null Hypothesis.
